<a href="https://colab.research.google.com/github/anupriya123-hub/Anupriya-Biopyhton-Pipeline/blob/main/Anupriya_Shandil_Biopython_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Biopython pipeline Assignment
#Anupriya Shandil
# Assigned IDs:
# Nucleotide accession: NM_001301717
# PDB ID: 2XJ1

NUC_ACCESSION = "NM_001301717"
PDB_ID = "2XJ1"
OUTPUT_DIR = "/content/output_files"

# -------------------------
# Imports & setup
# -------------------------
def install_requirements():
    import subprocess, sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", "biopython", "logomaker", "requests"])

try:
    import Bio
except ImportError:
    install_requirements()

import os
from pathlib import Path
os.makedirs(OUTPUT_DIR, exist_ok=True)

from Bio import Entrez, SeqIO, pairwise2
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.PDB import PDBList, PDBParser, PPBuilder
from Bio.SeqFeature import SeqFeature, FeatureLocation
import matplotlib.pyplot as plt
import numpy as np



def fetch_nucleotide_record(acc):
    h = Entrez.efetch(db="nucleotide", id=acc, rettype="gb", retmode="text")
    rec = SeqIO.read(h, "genbank")
    h.close()
    return rec

def fetch_pdb_file(pdb_id, outdir=OUTPUT_DIR):
    pdbl = PDBList()
    return pdbl.retrieve_pdb_file(pdb_id, pdir=outdir, file_format="pdb")

def extract_sequences_from_pdb(pdb_path):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("PDBSTRUCT", pdb_path)
    ppb = PPBuilder()
    seq_records = []
    for model in structure:
        for chain in model:
            peptides = ppb.build_peptides(chain)
            seq = "".join([str(p.get_sequence()) for p in peptides])
            if seq:
                seq_records.append(SeqRecord(Seq(seq), id=f"{structure.id}_{chain.id}", description=f"chain {chain.id}"))
    return seq_records

def gc_content(seq):
    s = str(seq).upper()
    return (s.count("G")+s.count("C"))/len(s)*100 if len(s)>0 else 0

def sliding_gc(seq, window=200, step=50):
    s = str(seq).upper()
    L = len(s)
    values = []
    for i in range(0, max(1, L-window+1), step):
        values.append((i, gc_content(s[i:i+window])))
    return values

def find_orfs(dna_seq, min_prot_len=50):
    s = Seq(str(dna_seq).upper())
    results = []
    stops = {"TAA","TAG","TGA"}
    def scan(seq, strand):
        seqstr = str(seq)
        L = len(seqstr)
        for frame in range(3):
            i = frame
            while i+3 <= L:
                codon = seqstr[i:i+3]
                if codon=="ATG":
                    j=i; prot=[]
                    while j+3<=L:
                        c=seqstr[j:j+3]
                        if c in stops: break
                        prot.append(str(Seq(c).translate()))
                        j+=3
                    if len(prot)>=min_prot_len:
                        start=i if strand==1 else L-(j+3)
                        end=j+3 if strand==1 else L-i
                        results.append({
                            "strand": strand,
                            "frame": frame if strand==1 else -frame-1,
                            "start": start,
                            "end": end,
                            "protein_seq": "".join(prot)
                        })
                i+=3
    scan(s,1); scan(s.reverse_complement(),-1)
    return results

def translate_full(dna_seq):
    return str(Seq(str(dna_seq)).translate(to_stop=False))

def pairwise_align(prot1, prot2, top=1):
    return pairwise2.align.globalxx(prot1, prot2)[:top]

def write_alignment_text(alignments, path):
    from Bio.pairwise2 import format_alignment
    with open(path,"w") as f:
        for a in alignments:
            f.write(format_alignment(*a)+"\n")

def plot_gc_windows(windows, path):
    x=[p for p,_ in windows]; y=[v for _,v in windows]
    plt.figure(figsize=(10,3))
    plt.plot(x,y,"-o")
    plt.xlabel("Position (nt)"); plt.ylabel("GC%"); plt.title("Sliding GC content")
    plt.savefig(path); plt.close()

def annotate_orfs(record, orfs, path):
    feats = list(record.features)
    for i,orf in enumerate(orfs):
        loc=FeatureLocation(orf["start"],orf["end"],strand=1 if orf["strand"]==1 else -1)
        quals={"product":[f"ORF_{i+1}"],"translation":[orf["protein_seq"]]}
        feats.append(SeqFeature(loc,"CDS",quals))
    record.features=feats
    SeqIO.write(record,path,"genbank")


def plot_alignment(aln, outpath):
    """Visualize alignment as a match/mismatch plot."""
    seqA, seqB, score, start, end = aln
    matches = ["|" if a==b else " " for a,b in zip(seqA, seqB)]
    plt.figure(figsize=(12,2))
    plt.imshow([[1 if m=="|" else 0 for m in matches]], cmap="Greens", aspect="auto")
    plt.yticks([])
    plt.xlabel("Alignment position")
    plt.title("Alignment Matches (green = match)")
    plt.savefig(outpath); plt.close()

def codon_usage_plot(seq, outpath):
    """Plot codon frequency bar chart."""
    seq=str(seq).upper()
    codons=[seq[i:i+3] for i in range(0,len(seq)-2,3)]
    freq={}
    for c in codons:
        if len(c)==3: freq[c]=freq.get(c,0)+1
    items=sorted(freq.items())
    labels=[i[0] for i in items]; values=[i[1] for i in items]
    plt.figure(figsize=(14,4))
    plt.bar(labels,values)
    plt.xticks(rotation=90)
    plt.title("Codon Usage")
    plt.tight_layout()
    plt.savefig(outpath); plt.close()

def aa_composition_plot(prot, outpath):
    """Plot amino acid composition bar chart."""
    freq={}
    for aa in prot:
        freq[aa]=freq.get(aa,0)+1
    items=sorted(freq.items())
    labels=[i[0] for i in items]; values=[i[1] for i in items]
    plt.figure(figsize=(10,4))
    plt.bar(labels,values)
    plt.title("Amino Acid Composition")
    plt.tight_layout()
    plt.savefig(outpath); plt.close()

# -------------------------
# Main pipeline
# -------------------------
def main():
    print("Fetching nucleotide...")
    nb=fetch_nucleotide_record(NUC_ACCESSION)
    SeqIO.write(nb, Path(OUTPUT_DIR)/f"{NUC_ACCESSION}.fasta","fasta")

    print("Analyzing GC content...")
    gcw=sliding_gc(nb.seq,200)
    plot_gc_windows(gcw, Path(OUTPUT_DIR)/f"{NUC_ACCESSION}_gc.png")

    print("Finding ORFs...")
    orfs=find_orfs(nb.seq,50)
    print("ORFs found:",len(orfs))

    print("Translating sequence...")
    translated=translate_full(nb.seq)
    tr_record=SeqRecord(Seq(translated),id="translated")
    SeqIO.write(tr_record, Path(OUTPUT_DIR)/f"{NUC_ACCESSION}_translated.fasta","fasta")

    print("Codon usage & amino acid composition...")
    codon_usage_plot(nb.seq, Path(OUTPUT_DIR)/"codon_usage.png")
    aa_composition_plot(translated, Path(OUTPUT_DIR)/"aa_composition.png")

    print("Fetching PDB...")
    pdb_path=fetch_pdb_file(PDB_ID,OUTPUT_DIR)
    pdb_seqs=extract_sequences_from_pdb(pdb_path)
    if pdb_seqs:
        SeqIO.write(pdb_seqs,Path(OUTPUT_DIR)/f"{PDB_ID}_chains.fasta","fasta")
        aln=pairwise_align(str(tr_record.seq),str(pdb_seqs[0].seq))[0]
        write_alignment_text([aln], Path(OUTPUT_DIR)/"alignment.txt")
        plot_alignment(aln, Path(OUTPUT_DIR)/"alignment.png")

    print("Annotating ORFs...")
    annotate_orfs(nb,orfs,Path(OUTPUT_DIR)/f"{NUC_ACCESSION}_annotated.gb")

    print("Pipeline finished. Outputs in",OUTPUT_DIR)

if __name__=="__main__":
    main()


/usr/local/lib/python3.12/dist-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


Fetching nucleotide...
Analyzing GC content...
Finding ORFs...
ORFs found: 17
Translating sequence...
Codon usage & amino acid composition...


/usr/local/lib/python3.12/dist-packages/Bio/Seq.py:2879: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


Fetching PDB...
Annotating ORFs...
Pipeline finished. Outputs in /content/output_files
